# Exploratory Data Analysis (EDA) - Step by Step
## Retail Sales Dataset

This notebook provides a comprehensive, step-by-step analysis of a retail sales dataset containing 62,884 transactions across 8 product categories, 58 stores, and 9 countries (2016–2021).

## 1. Import Libraries

In [ ]:
# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Settings
import warnings
from datetime import datetime

warnings.filterwarnings('ignore')
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Libraries imported successfully!")

## 2. Load Data

In [ ]:
# Load the Excel file
file_path = '../Data/Retail Sales Dataset.xlsx'  # Update this path if needed

df = pd.read_excel(file_path)

print(f"✓ Data loaded successfully!")
print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")

## 3. First Look at the Data

In [ ]:
# Display first few rows
df.head()

In [ ]:
# Display last few rows
df.tail()

In [ ]:
# Display random sample
df.sample(10)

## 4. Basic Information

In [ ]:
# Dataset shape
print(f"Number of rows: {df.shape[0]:,}")
print(f"Number of columns: {df.shape[1]}")

In [ ]:
# Column names
print("Column Names:")
for i, col in enumerate(df.columns, 1):
    print(f"{i}. {col}")

In [ ]:
# Data types and non-null counts
df.info()

In [ ]:
# Memory usage
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

## 5. Feature Engineering

Create calculated fields for deeper analysis.

In [ ]:
# Create calculated columns
df['LineTotal'] = df['Quantity'] * df['ProductPrice']
df['LineCost'] = df['Quantity'] * df['ProductCost']
df['Profit'] = df['LineTotal'] - df['LineCost']
df['ProfitMargin'] = (df['Profit'] / df['LineTotal'] * 100).round(2)
df['DeliveryDays'] = (df['DeliveryDate'] - df['OrderDate']).dt.days
df['OrderYear'] = df['OrderDate'].dt.year
df['OrderMonth'] = df['OrderDate'].dt.month
df['OrderDayOfWeek'] = df['OrderDate'].dt.day_name()

print("✓ Feature engineering complete!")
print(f"New columns added: LineTotal, LineCost, Profit, ProfitMargin, DeliveryDays, OrderYear, OrderMonth, OrderDayOfWeek")
print(f"Updated shape: {df.shape[0]:,} rows × {df.shape[1]} columns")

## 6. Missing Values Analysis

In [ ]:
# Calculate missing values
missing_data = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df) * 100).round(2)
})

# Show only columns with missing values
missing_data = missing_data[missing_data['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False)

if len(missing_data) > 0:
    print("Columns with missing values:")
    display(missing_data)
else:
    print("✓ No missing values found!")

In [ ]:
# Visualize missing values
if len(missing_data) > 0:
    plt.figure(figsize=(10, 4))
    plt.barh(missing_data['Column'], missing_data['Missing_Percentage'], color='coral')
    plt.xlabel('Missing Percentage (%)', fontsize=12)
    plt.ylabel('Column', fontsize=12)
    plt.title('Missing Values by Column', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

## 7. Numerical Features Analysis

In [ ]:
# Identify key numerical columns for analysis
key_numerical = ['Quantity', 'ProductCost', 'ProductPrice', 'LineTotal', 'Profit', 'ProfitMargin', 'DeliveryDays', 'StoreSqMeters']

print(f"Key numerical columns for analysis ({len(key_numerical)}):")
for col in key_numerical:
    print(f"  • {col}")

In [ ]:
# Descriptive statistics
df[key_numerical].describe().round(2)

In [ ]:
# Distribution of key numerical features (Histograms)
fig, axes = plt.subplots(2, 4, figsize=(20, 10))
axes = axes.flatten()

for idx, col in enumerate(key_numerical):
    data = df[col].dropna()
    axes[idx].hist(data, bins=30, edgecolor='black', color='skyblue')
    axes[idx].set_title(f'Distribution of {col}', fontsize=10, fontweight='bold')
    axes[idx].set_xlabel(col)
    axes[idx].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
# Box plots to identify outliers
fig, axes = plt.subplots(2, 4, figsize=(20, 10))
axes = axes.flatten()

for idx, col in enumerate(key_numerical):
    data = df[col].dropna()
    axes[idx].boxplot(data)
    axes[idx].set_title(f'Box Plot: {col}', fontsize=10, fontweight='bold')
    axes[idx].set_ylabel(col)

plt.tight_layout()
plt.show()

## 8. Correlation Analysis

In [ ]:
# Correlation matrix
correlation = df[key_numerical].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix of Key Numerical Features', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Find highly correlated pairs (correlation > 0.7 or < -0.7)
high_corr_pairs = []

for i in range(len(correlation.columns)):
    for j in range(i+1, len(correlation.columns)):
        if abs(correlation.iloc[i, j]) > 0.7:
            high_corr_pairs.append({
                'Feature 1': correlation.columns[i],
                'Feature 2': correlation.columns[j],
                'Correlation': round(correlation.iloc[i, j], 3)
            })

if high_corr_pairs:
    print("Highly correlated feature pairs (|correlation| > 0.7):")
    display(pd.DataFrame(high_corr_pairs))
else:
    print("No highly correlated pairs found.")

## 9. Categorical Features Analysis

In [ ]:
# Key categorical columns
categorical_cols = ['CustomerGender', 'CustomerCountry', 'CustomerContinent',
                    'StoreCountry', 'ProductCategory', 'ProductSubcategory',
                    'ProductBrand', 'ProductColor']

print(f"Key categorical columns ({len(categorical_cols)}):")
for col in categorical_cols:
    print(f"  • {col}: {df[col].nunique()} unique values")

In [ ]:
# Value counts for each categorical column
for col in categorical_cols:
    print(f"\n{'='*60}")
    print(f"{col} - Value Counts:")
    print('='*60)
    print(df[col].value_counts().head(10))

In [ ]:
# Visualize categorical distributions
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Gender
gender_counts = df['CustomerGender'].value_counts()
axes[0, 0].pie(gender_counts.values, labels=gender_counts.index, autopct='%1.1f%%', startangle=90)
axes[0, 0].set_title('Customer Gender Split', fontweight='bold')

# Customer Country
country_counts = df.groupby('CustomerCountry')['OrderNumber'].nunique().sort_values(ascending=True)
axes[0, 1].barh(country_counts.index, country_counts.values, color='steelblue')
axes[0, 1].set_title('Orders by Customer Country', fontweight='bold')
axes[0, 1].set_xlabel('Number of Orders')

# Product Category
cat_counts = df['ProductCategory'].value_counts()
axes[1, 0].barh(range(len(cat_counts)), cat_counts.values, color='coral')
axes[1, 0].set_yticks(range(len(cat_counts)))
axes[1, 0].set_yticklabels(cat_counts.index)
axes[1, 0].set_title('Transaction Count by Category', fontweight='bold')
axes[1, 0].set_xlabel('Transactions')
axes[1, 0].invert_yaxis()

# Top Brands
brand_counts = df['ProductBrand'].value_counts().head(10)
axes[1, 1].barh(range(len(brand_counts)), brand_counts.values, color='teal')
axes[1, 1].set_yticks(range(len(brand_counts)))
axes[1, 1].set_yticklabels(brand_counts.index)
axes[1, 1].set_title('Top 10 Brands by Transaction Count', fontweight='bold')
axes[1, 1].set_xlabel('Transactions')
axes[1, 1].invert_yaxis()

plt.tight_layout()
plt.show()

## 10. Retail-Specific Analysis

### 10.1 Revenue Analysis

In [ ]:
# Overall revenue metrics
total_revenue = df['LineTotal'].sum()
total_cost = df['LineCost'].sum()
total_profit = df['Profit'].sum()
profit_margin = (total_profit / total_revenue * 100)

order_totals = df.groupby('OrderNumber')['LineTotal'].sum()
avg_order_value = order_totals.mean()
median_order_value = order_totals.median()
total_orders = df['OrderNumber'].nunique()

print("REVENUE METRICS")
print("="*50)
print(f"Total Revenue: ${total_revenue:,.2f}")
print(f"Total Cost: ${total_cost:,.2f}")
print(f"Total Profit: ${total_profit:,.2f}")
print(f"Profit Margin: {profit_margin:.1f}%")
print(f"Average Order Value: ${avg_order_value:,.2f}")
print(f"Median Order Value: ${median_order_value:,.2f}")
print(f"Total Orders: {total_orders:,}")
print(f"Total Transactions: {len(df):,}")
print(f"Total Customers: {df['CustomerID'].nunique():,}")
print(f"Total Products: {df['ProductID'].nunique():,}")

In [ ]:
# Revenue distribution (order-level)
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histogram
axes[0].hist(order_totals[order_totals < 10000], bins=50, edgecolor='black', color='green', alpha=0.7)
axes[0].set_xlabel('Order Value ($)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Order Values', fontweight='bold')
axes[0].axvline(avg_order_value, color='red', linestyle='--', label=f'Mean: ${avg_order_value:,.2f}')
axes[0].legend()

# Box plot
axes[1].boxplot(order_totals, vert=True)
axes[1].set_ylabel('Order Value ($)')
axes[1].set_title('Order Value Box Plot', fontweight='bold')

plt.tight_layout()
plt.show()

### 10.2 Revenue Trend Over Time

In [ ]:
# Revenue by year
yearly_revenue = df.groupby('OrderYear').agg(
    Revenue=('LineTotal', 'sum'),
    Orders=('OrderNumber', 'nunique'),
    Profit=('Profit', 'sum')
).round(2)

print("REVENUE BY YEAR")
print("="*60)
display(yearly_revenue)

In [ ]:
# Visualize yearly revenue trend
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Revenue trend
axes[0].plot(yearly_revenue.index, yearly_revenue['Revenue'], marker='o', linewidth=2, color='steelblue')
axes[0].fill_between(yearly_revenue.index, yearly_revenue['Revenue'], alpha=0.2, color='steelblue')
axes[0].set_title('Revenue Trend by Year', fontweight='bold')
axes[0].set_xlabel('Year')
axes[0].set_ylabel('Revenue ($)')
axes[0].grid(True, alpha=0.3)

# Orders trend
axes[1].bar(yearly_revenue.index, yearly_revenue['Orders'], color='coral')
axes[1].set_title('Order Count by Year', fontweight='bold')
axes[1].set_xlabel('Year')
axes[1].set_ylabel('Number of Orders')

plt.tight_layout()
plt.show()

In [ ]:
# Monthly revenue pattern
monthly_revenue = df.groupby('OrderMonth')['LineTotal'].sum()
month_names = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

plt.figure(figsize=(12, 5))
plt.bar(month_names, monthly_revenue.values, color='steelblue')
plt.title('Revenue by Month (All Years Combined)', fontweight='bold')
plt.xlabel('Month')
plt.ylabel('Revenue ($)')
plt.tight_layout()
plt.show()

### 10.3 Category Analysis

In [ ]:
# Revenue by category
category_revenue = df.groupby('ProductCategory').agg(
    Total_Revenue=('LineTotal', 'sum'),
    Total_Profit=('Profit', 'sum'),
    Avg_Price=('ProductPrice', 'mean'),
    Order_Count=('OrderNumber', 'nunique'),
    Qty_Sold=('Quantity', 'sum')
).round(2)

category_revenue = category_revenue.sort_values('Total_Revenue', ascending=False)

print("REVENUE BY CATEGORY")
print("="*80)
display(category_revenue)

In [ ]:
# Visualize category performance
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# Total revenue
category_revenue['Total_Revenue'].plot(kind='barh', ax=axes[0], color='steelblue')
axes[0].set_title('Total Revenue by Category', fontweight='bold')
axes[0].set_xlabel('Revenue ($)')

# Profit
category_revenue['Total_Profit'].plot(kind='barh', ax=axes[1], color='green')
axes[1].set_title('Total Profit by Category', fontweight='bold')
axes[1].set_xlabel('Profit ($)')

# Quantity sold
category_revenue['Qty_Sold'].plot(kind='barh', ax=axes[2], color='coral')
axes[2].set_title('Quantity Sold by Category', fontweight='bold')
axes[2].set_xlabel('Quantity')

plt.tight_layout()
plt.show()

### 10.4 Brand Analysis

In [ ]:
# Top 10 brands by revenue
brand_revenue = df.groupby('ProductBrand').agg(
    Total_Revenue=('LineTotal', 'sum'),
    Total_Profit=('Profit', 'sum'),
    Order_Count=('OrderNumber', 'nunique'),
    Products=('ProductID', 'nunique')
).round(2).sort_values('Total_Revenue', ascending=False).head(10)

print("TOP 10 BRANDS BY REVENUE")
print("="*70)
display(brand_revenue)

In [ ]:
# Visualize top brands
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Revenue
brand_revenue['Total_Revenue'].plot(kind='barh', ax=axes[0], color='darkblue')
axes[0].set_title('Top 10 Brands by Revenue', fontweight='bold')
axes[0].set_xlabel('Revenue ($)')

# Order count
brand_revenue['Order_Count'].plot(kind='barh', ax=axes[1], color='darkgreen')
axes[1].set_title('Top 10 Brands by Order Count', fontweight='bold')
axes[1].set_xlabel('Number of Orders')

plt.tight_layout()
plt.show()

### 10.5 Geographic Analysis

In [ ]:
# Revenue by customer country
country_revenue = df.groupby('CustomerCountry').agg(
    Total_Revenue=('LineTotal', 'sum'),
    Order_Count=('OrderNumber', 'nunique'),
    Customers=('CustomerID', 'nunique')
).round(2).sort_values('Total_Revenue', ascending=False)

print("REVENUE BY CUSTOMER COUNTRY")
print("="*70)
display(country_revenue)

In [ ]:
# Visualize geographic performance
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Revenue by country
country_revenue['Total_Revenue'].plot(kind='barh', ax=axes[0], color='darkblue')
axes[0].set_title('Revenue by Customer Country', fontweight='bold')
axes[0].set_xlabel('Revenue ($)')
axes[0].set_ylabel('Country')

# Customer count by country
country_revenue['Customers'].plot(kind='barh', ax=axes[1], color='teal')
axes[1].set_title('Customer Count by Country', fontweight='bold')
axes[1].set_xlabel('Customers')
axes[1].set_ylabel('Country')

plt.tight_layout()
plt.show()

In [ ]:
# Store channel analysis (Online vs Physical)
store_type = df.copy()
store_type['Channel'] = store_type['StoreCountry'].apply(lambda x: 'Online' if x == 'Online' else 'Physical Store')

channel_analysis = store_type.groupby('Channel').agg(
    Revenue=('LineTotal', 'sum'),
    Orders=('OrderNumber', 'nunique'),
    Transactions=('TransactionID', 'count')
).round(2)

print("ONLINE VS PHYSICAL STORE")
print("="*60)
display(channel_analysis)

print(f"\nOnline share of revenue: {channel_analysis.loc['Online', 'Revenue'] / channel_analysis['Revenue'].sum() * 100:.1f}%")

In [ ]:
# Visualize online vs physical
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Revenue split
axes[0].pie(channel_analysis['Revenue'].values, labels=channel_analysis.index,
            autopct='%1.1f%%', startangle=90, colors=['#ff9f43', '#54a0ff'])
axes[0].set_title('Revenue: Online vs Physical', fontweight='bold')

# Order split
axes[1].pie(channel_analysis['Orders'].values, labels=channel_analysis.index,
            autopct='%1.1f%%', startangle=90, colors=['#ff9f43', '#54a0ff'])
axes[1].set_title('Orders: Online vs Physical', fontweight='bold')

plt.tight_layout()
plt.show()

### 10.6 Customer Demographics

In [ ]:
# Gender analysis
gender_analysis = df.groupby('CustomerGender').agg(
    Revenue=('LineTotal', 'sum'),
    Orders=('OrderNumber', 'nunique'),
    Customers=('CustomerID', 'nunique'),
    Avg_Order_Value=('LineTotal', 'mean')
).round(2)

print("GENDER ANALYSIS")
print("="*60)
display(gender_analysis)

In [ ]:
# Visualize gender split
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Revenue by gender
axes[0].pie(gender_analysis['Revenue'].values, labels=gender_analysis.index,
            autopct='%1.1f%%', startangle=90, colors=['#ff6b6b', '#54a0ff'])
axes[0].set_title('Revenue by Gender', fontweight='bold')

# Customer count by gender
axes[1].bar(gender_analysis.index, gender_analysis['Customers'], color=['#ff6b6b', '#54a0ff'])
axes[1].set_title('Customer Count by Gender', fontweight='bold')
axes[1].set_ylabel('Customers')

plt.tight_layout()
plt.show()

### 10.7 Product Color Analysis

In [ ]:
# Top product colors by revenue
color_revenue = df.groupby('ProductColor')['LineTotal'].sum().sort_values(ascending=False).head(10)

print("TOP 10 PRODUCT COLORS BY REVENUE")
print("="*50)
for color, rev in color_revenue.items():
    print(f"  {color}: ${rev:,.2f}")

In [ ]:
# Visualize product colors
plt.figure(figsize=(12, 5))
color_revenue.plot(kind='bar', color=['#333333','#e0e0e0','#c0c0c0','#808080','#8B4513',
                                       '#4169E1','#DC143C','#228B22','#FFD700','#FF69B4'])
plt.title('Top 10 Product Colors by Revenue', fontweight='bold')
plt.xlabel('Color')
plt.ylabel('Revenue ($)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### 10.8 Delivery Analysis

In [ ]:
# Delivery days analysis
delivery = df['DeliveryDays'].dropna()

print("DELIVERY ANALYSIS")
print("="*50)
print(f"Transactions with delivery data: {len(delivery):,} ({len(delivery)/len(df)*100:.1f}%)")
print(f"Transactions without delivery data: {df['DeliveryDays'].isna().sum():,} ({df['DeliveryDays'].isna().sum()/len(df)*100:.1f}%)")
print(f"\nAverage Delivery Days: {delivery.mean():.1f}")
print(f"Median Delivery Days: {delivery.median():.1f}")
print(f"Min Delivery Days: {delivery.min():.0f}")
print(f"Max Delivery Days: {delivery.max():.0f}")

In [ ]:
# Visualize delivery days
plt.figure(figsize=(12, 5))
plt.hist(delivery, bins=range(0, int(delivery.max()) + 2), edgecolor='black', color='teal', alpha=0.7)
plt.axvline(delivery.mean(), color='red', linestyle='--', label=f'Mean: {delivery.mean():.1f} days')
plt.xlabel('Delivery Days')
plt.ylabel('Frequency')
plt.title('Distribution of Delivery Days', fontweight='bold')
plt.legend()
plt.tight_layout()
plt.show()

### 10.9 Subcategory Deep Dive

In [ ]:
# Top 10 subcategories by revenue
subcat_revenue = df.groupby('ProductSubcategory').agg(
    Revenue=('LineTotal', 'sum'),
    Orders=('OrderNumber', 'nunique')
).round(2).sort_values('Revenue', ascending=False).head(10)

print("TOP 10 SUBCATEGORIES BY REVENUE")
print("="*60)
display(subcat_revenue)

In [ ]:
# Visualize top subcategories
plt.figure(figsize=(14, 6))
subcat_revenue['Revenue'].plot(kind='barh', color='steelblue')
plt.title('Top 10 Subcategories by Revenue', fontweight='bold')
plt.xlabel('Revenue ($)')
plt.ylabel('Subcategory')
plt.tight_layout()
plt.show()

## 11. Outlier Detection

In [ ]:
# Detect outliers using IQR method
print("OUTLIER DETECTION (IQR Method)")
print("="*70)

outlier_cols = ['Quantity', 'ProductCost', 'ProductPrice', 'LineTotal', 'Profit', 'DeliveryDays']
outlier_summary = []

for col in outlier_cols:
    data = df[col].dropna()
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = data[(data < lower_bound) | (data > upper_bound)]
    outlier_count = len(outliers)
    outlier_percentage = (outlier_count / len(data)) * 100

    if outlier_count > 0:
        outlier_summary.append({
            'Column': col,
            'Outlier_Count': outlier_count,
            'Outlier_%': round(outlier_percentage, 2),
            'Lower_Bound': round(lower_bound, 2),
            'Upper_Bound': round(upper_bound, 2)
        })

if outlier_summary:
    outlier_df = pd.DataFrame(outlier_summary)
    display(outlier_df)
else:
    print("✓ No significant outliers detected!")

## 12. Data Quality Report

In [ ]:
# Comprehensive data quality report
quality_report = []

for col in df.columns:
    missing_count = df[col].isnull().sum()
    missing_pct = (missing_count / len(df)) * 100
    unique_count = df[col].nunique()
    dtype = df[col].dtype

    quality_report.append({
        'Column': col,
        'Data_Type': str(dtype),
        'Missing_Count': missing_count,
        'Missing_%': round(missing_pct, 2),
        'Unique_Values': unique_count,
        'Completeness_%': round(100 - missing_pct, 2)
    })

quality_df = pd.DataFrame(quality_report)

print("DATA QUALITY REPORT")
print("="*100)
display(quality_df)

## 13. Key Insights Summary

In [ ]:
# Generate automated insights
print("KEY INSIGHTS SUMMARY")
print("="*70)

print(f"\n\U0001f4ca DATASET OVERVIEW:")
print(f"  \u2022 Total Transactions: {len(df):,}")
print(f"  \u2022 Total Features: {len(df.columns)}")
print(f"  \u2022 Date Range: {df['OrderDate'].min().strftime('%Y-%m-%d')} to {df['OrderDate'].max().strftime('%Y-%m-%d')}")
print(f"  \u2022 Countries: {df['CustomerCountry'].nunique()} customer countries, {df['StoreCountry'].nunique()} store locations")

print(f"\n\U0001f4b0 REVENUE INSIGHTS:")
print(f"  \u2022 Total Revenue: ${df['LineTotal'].sum():,.2f}")
print(f"  \u2022 Total Profit: ${df['Profit'].sum():,.2f} ({df['Profit'].sum()/df['LineTotal'].sum()*100:.1f}% margin)")
print(f"  \u2022 Average Order Value: ${order_totals.mean():,.2f}")
print(f"  \u2022 Peak Year: {df.groupby('OrderYear')['LineTotal'].sum().idxmax()} (${df.groupby('OrderYear')['LineTotal'].sum().max():,.2f})")

top_cat = df.groupby('ProductCategory')['LineTotal'].sum().idxmax()
top_cat_rev = df.groupby('ProductCategory')['LineTotal'].sum().max()
print(f"\n\U0001f4e6 CATEGORY INSIGHTS:")
print(f"  \u2022 Top Category: {top_cat} (${top_cat_rev:,.2f})")
print(f"  \u2022 8 product categories, 32 subcategories, {df['ProductID'].nunique():,} unique products")

top_brand = df.groupby('ProductBrand')['LineTotal'].sum().idxmax()
top_brand_rev = df.groupby('ProductBrand')['LineTotal'].sum().max()
print(f"\n\U0001f3f7 BRAND INSIGHTS:")
print(f"  \u2022 Top Brand: {top_brand} (${top_brand_rev:,.2f})")
print(f"  \u2022 Total Brands: {df['ProductBrand'].nunique()}")

top_country = df.groupby('CustomerCountry')['LineTotal'].sum().idxmax()
top_country_rev = df.groupby('CustomerCountry')['LineTotal'].sum().max()
print(f"\n\U0001f30d GEOGRAPHIC INSIGHTS:")
print(f"  \u2022 Top Country: {top_country} (${top_country_rev:,.2f})")
print(f"  \u2022 Online vs Physical: {df[df['StoreCountry']=='Online']['LineTotal'].sum()/df['LineTotal'].sum()*100:.1f}% online revenue")

print(f"\n\u26a0\ufe0f DATA QUALITY:")
print(f"  \u2022 DeliveryDate is 79.06% missing (49,719 transactions)")
print(f"  \u2022 CustomerStateCode has 30 missing values (0.05%)")
print(f"  \u2022 Average delivery time: {df['DeliveryDays'].dropna().mean():.1f} days (for records with data)")

print("\n\u2705 Analysis Complete!")

## 14. Next Steps & Recommendations

Based on the EDA, consider these next steps:

1. **Investigate Missing Delivery Dates**: 79% of transactions lack delivery dates — determine if these are in-store pickups vs data gaps
2. **Revenue Forecasting**: Build time series models using the 5-year trend data (2016–2021)
3. **Customer Segmentation**: Use RFM analysis to identify high-value customer segments
4. **Product Profitability**: Analyze margin differences across categories and brands
5. **Geographic Expansion**: Evaluate underperforming markets for growth opportunities
6. **Online vs Physical**: Deep-dive into channel performance to optimize omnichannel strategy
7. **Seasonal Patterns**: Investigate the revenue dip in April and peak in February/December